In [1]:
import os
import pandas as pd
from scipy import spatial
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv("../../../../../../../apis/.env")
api_key = os.getenv("OPENAI_API_KEY")
unmasked_chars = 8
masked_key = api_key[:unmasked_chars] + '*' * (len(api_key) - unmasked_chars*2) + api_key[-unmasked_chars:]
print(f"API key: {masked_key}")
client = OpenAI(api_key=api_key)
print("Cliente inicializado como",client)

API key: sk-proj-****************************************************************************************************************************************************-amA_5sA
Cliente inicializado como <openai.OpenAI object at 0x0000021664BC5ED0>


## 1. Ejemplos básicos de cálculo de distancia con embeddings

In [2]:
ejemplos_experiencia = pd.read_pickle("../pkl/df_experiencia.pkl")
display(ejemplos_experiencia)
print(ejemplos_experiencia.puesto[1])

,empresa,puesto,periodo,fec_inicio,fec_final,duracion
0,Autónomo,Comercial de automoviles,202401,2024-01-01,2024-12-07,11
1,Mercadona,Vendedor/a de puesto de mercado,202310-202404,2023-10-01,2024-04-01,6
2,AGRISOLUTIONS,AUXILIAR DE MANTENIMIENTO INDUSTRIAL,202001-202401,2020-01-01,2024-01-01,48
3,GASTROTEKA ORDIZIA 1990,Camarero/a de barra,202303-202309,2023-03-01,2023-09-01,6
4,ZEREGUIN ZERBITZUAK,limpieza industrial,202012-202305,2020-12-01,2023-05-01,29
5,Bellota Herramientas,Personal de mantenimiento,202005-202011,2020-05-01,2020-11-01,6


Vendedor/a de puesto de mercado


In [5]:
client = OpenAI()
puesto_vendedor = ejemplos_experiencia.puesto[1]

response = client.embeddings.create(
    input=puesto_vendedor,
    model="text-embedding-3-small"
)
emb_puesto_vendedor = response.data[0].embedding
print(f'Texto: {puesto_vendedor}\nEmbeddings ({len(emb_puesto_vendedor)}): {emb_puesto_vendedor[:10]}...')

Texto: Vendedor/a de puesto de mercado
Embeddings (1536): [-0.006109286565333605, -0.01615688018500805, 0.02458987757563591, 0.0013343609170988202, -0.04200134426355362, 0.015196849592030048, 0.010587611235678196, 0.03497566282749176, -0.015262306667864323, -0.031200997531414032]...


In [6]:
puesto_camarero = ejemplos_experiencia.puesto[3]

response = client.embeddings.create(
    input=puesto_camarero,
    model="text-embedding-3-small"
)
emb_puesto_camarero = response.data[0].embedding
print(f'Texto: {puesto_camarero}\nEmbeddings ({len(emb_puesto_camarero)}): {emb_puesto_camarero[:10]}...')

Texto: Camarero/a de barra
Embeddings (1536): [-0.035160087049007416, -0.0017518880777060986, -0.006896876264363527, -0.040239546447992325, -0.024628372862935066, 0.000213889084989205, 4.456970600585919e-06, 0.047462623566389084, -0.02062072791159153, -0.03217765688896179]...


In [7]:
oferta_cajero = "Cajero supermercado Dia"

response = client.embeddings.create(
    input=oferta_cajero,
    model="text-embedding-3-small"
)
emb_oferta_cajero = response.data[0].embedding
print(f'Texto: {oferta_cajero}\nEmbeddings ({len(emb_oferta_cajero)}): {emb_oferta_cajero[:10]}...')

Texto: Cajero supermercado Dia
Embeddings (1536): [-0.0045319367200136185, -0.04426201060414314, -0.0222327820956707, -0.015300587750971317, 0.008034787140786648, 0.011099428869783878, 0.03736374154686928, 0.07590357959270477, -0.020332932472229004, -0.03946714848279953]...


In [8]:
dist_min = spatial.distance.cosine(emb_oferta_cajero, emb_oferta_cajero)
print(f"Distancia mínima: {dist_min:.3f}")
dist_ven = spatial.distance.cosine(emb_puesto_vendedor, emb_oferta_cajero)
print(f"Distancia entre el puesto de vendedor y la oferta de cajero: {dist_ven:.3f}")
dist_cam = spatial.distance.cosine(emb_puesto_camarero, emb_oferta_cajero)
print(f"Distancia entre el puesto de camarero y la oferta de cajero: {dist_cam:.3f}")


Distancia mínima: 0.000
Distancia entre el puesto de vendedor y la oferta de cajero: 0.557
Distancia entre el puesto de camarero y la oferta de cajero: 0.587


## 2. Análisis de cálculo de distancias para el CV completo

In [9]:
ejemplos_experiencia['embeddings'] = ejemplos_experiencia['puesto'].apply(lambda puesto: client.embeddings.create(input=puesto, model="text-embedding-3-small").data[0].embedding)
display(ejemplos_experiencia)

,empresa,puesto,periodo,fec_inicio,fec_final,duracion,embeddings
0,Autónomo,Comercial de automoviles,202401,2024-01-01,2024-12-07,11,"[0.015070287510752678, 0.0029741383623331785, ..."
1,Mercadona,Vendedor/a de puesto de mercado,202310-202404,2023-10-01,2024-04-01,6,"[-0.006109286565333605, -0.01615688018500805, ..."
2,AGRISOLUTIONS,AUXILIAR DE MANTENIMIENTO INDUSTRIAL,202001-202401,2020-01-01,2024-01-01,48,"[0.00385109125636518, 0.04469580203294754, 0.0..."
3,GASTROTEKA ORDIZIA 1990,Camarero/a de barra,202303-202309,2023-03-01,2023-09-01,6,"[-0.035160087049007416, -0.0017518880777060986..."
4,ZEREGUIN ZERBITZUAK,limpieza industrial,202012-202305,2020-12-01,2023-05-01,29,"[0.003700299421325326, 0.0045193759724497795, ..."
5,Bellota Herramientas,Personal de mantenimiento,202005-202011,2020-05-01,2020-11-01,6,"[0.04391268640756607, 0.05462520197033882, 0.0..."


Calculamos la distancia entre la oferta "Cajero supermercado Dia" y cada uno de los puestos. Podemos observar que el modelo de embeddings de OpenAI es razonablemente bueno encontrando las relaciones semánticas entre textos como los del ejemplo. La experiencia que claramente tiene más relación es la que obtiene una distancia más baja. Para valorar la adecuación de los currículos a una oferta dada podríamos, obviamente, usar más datos tanto del CV como de la oferta, pero este ejemplo a pequeña escala demuestra la utilidad de los embeddings para discriminar puestos de trabajo relacionados entre sí:

In [10]:
oferta_cajero = "Cajero supermercado Dia"
response = client.embeddings.create(
    input=oferta_cajero,
    model="text-embedding-3-small"
)
emb_oferta_cajero = response.data[0].embedding

ejemplos_experiencia['distancia_oferta_cajero'] = ejemplos_experiencia['embeddings'].apply(lambda emb: spatial.distance.cosine(emb, emb_oferta_cajero))
ejemplos_experiencia.drop(columns=['embeddings'], inplace=True)
ejemplos_experiencia_sorted = ejemplos_experiencia.sort_values(by='distancia_oferta_cajero', ascending=True).copy()
display(ejemplos_experiencia_sorted)

,empresa,puesto,periodo,fec_inicio,fec_final,duracion,distancia_oferta_cajero
1,Mercadona,Vendedor/a de puesto de mercado,202310-202404,2023-10-01,2024-04-01,6,0.556915
3,GASTROTEKA ORDIZIA 1990,Camarero/a de barra,202303-202309,2023-03-01,2023-09-01,6,0.587302
2,AGRISOLUTIONS,AUXILIAR DE MANTENIMIENTO INDUSTRIAL,202001-202401,2020-01-01,2024-01-01,48,0.617411
0,Autónomo,Comercial de automoviles,202401,2024-01-01,2024-12-07,11,0.628034
5,Bellota Herramientas,Personal de mantenimiento,202005-202011,2020-05-01,2020-11-01,6,0.647794
4,ZEREGUIN ZERBITZUAK,limpieza industrial,202012-202305,2020-12-01,2023-05-01,29,0.701754


Guardamos el pickle para continuar usando este ejemplo en el siguiente bloque:

In [10]:
ejemplos_experiencia_sorted.to_pickle("../pkl/df_ejemplos_con_distancia.pkl")

## 3. Algoritmo de cálculo de puntuación

Experimentando con múltiples ficheros de datos, podríamos llegar a refinar una fórmula de cálculo de "puntuación" que se adapte a nuestro caso de uso, en función de las distancias simples calculadas con embeddings y los datos de cada experiencia (tiempo de permanencia en el puesto, antigüedad de la experiencia, etc.). Con suficientes datos, podríamos incluso entrenar nuestra propia red neuronal con embeddings para determinar la predictibilidad de un cierto cambio de puesto. Por ejemplo, parece relativamente asequible, con suficientes datos de currículos incluyendo fechas, conseguir "predecir" que un CV cuyas últimas dos experiencias sean "Vendedor de Planta" y "Analista de Pricing" sea más apropiado para un puesto con título "Jefe de Compras", que un CV con última experiencia "Jefe de Compras" a un puesto con título "Vendedor de Planta". Ese tipo de relaciones semánticas y causales específicas a una industria o a un ámbito muy específico es muy difícil de obtener con un modelo de lenguaje preentrenado, pero a día de hoy tenemos las herramientas que nos facilitan "refinar" (finetuning) cualquiera de esos grandes modelos sin un coste muy elevado, utilizando los datos que se adapten a nuestro específico caso de uso. 

<br>Para esta prueba de concepto, no disponemos de una amplia base de datos de currículos, por lo que definiremos un **sistema de puntuación simplificado basado exclusivamente en las distancias de embeddings, en la cantidad de experiencias previas y en la duración de las mismas**. No tendremos en cuenta factores muy importantes como la inferencia de causalidad y secuencialidad, así como detalles de los currículos y de la oferta de trabajo más allá de los títulos. 

<br>En cualquier caso, debe tenerse en cuenta que un sistema de análisis algorítmico sobre datos de CVs ha de usarse con suma cautela, debido al alto riesgo de obtener "falsos negativos" (https://es.wikipedia.org/wiki/Falso_positivo_y_falso_negativo): descartar un candidato potencialmente bueno, sin llegar a ver más datos que los de un fichero de texto. En este caso de uso, el riesgo de "falso positivo" (no descartar a un candidato no apropiado), no es tan crítico, dado que la revisión de datos de CVs es sólo una fase muy preliminar de un proceso de selección. En otras palabras, **el impacto en el negocio del "falso positivo" es hacer una entrevista de más, mientras que el impacto de un "falso negativo" es perder un buen candidato.**


In [11]:
ejemplos_experiencia_sorted = pd.read_pickle("../pkl/df_ejemplos_con_distancia.pkl")
ejemplos_experiencia_sorted.rename(columns={'distancia_oferta_cajero':'distancia'}, inplace=True)
display(ejemplos_experiencia_sorted)

,empresa,puesto,periodo,fec_inicio,fec_final,duracion,distancia
1,Mercadona,Vendedor/a de puesto de mercado,202310-202404,2023-10-01,2024-04-01,6,0.556915
3,GASTROTEKA ORDIZIA 1990,Camarero/a de barra,202303-202309,2023-03-01,2023-09-01,6,0.587302
2,AGRISOLUTIONS,AUXILIAR DE MANTENIMIENTO INDUSTRIAL,202001-202401,2020-01-01,2024-01-01,48,0.617411
0,Autónomo,Comercial de automoviles,202401,2024-01-01,2024-12-07,11,0.628034
5,Bellota Herramientas,Personal de mantenimiento,202005-202011,2020-05-01,2020-11-01,6,0.647790
4,ZEREGUIN ZERBITZUAK,limpieza industrial,202012-202305,2020-12-01,2023-05-01,29,0.701754


Algoritmo de puntuación:

In [12]:
def calcula_puntuacion(df, req_experience, positions_cap=4, min_dist_threshold=0.6, max_dist_threshold=0.7):
    """
    Calcula la puntuación de un CV a partir de su tabla de distancias (con respecto a un puesto dado) y duraciones. 

    Params:
    df (pandas.DataFrame): datos de un CV incluyendo diferentes experiencias incluyendo duracies y distancia previamente calculadas sobre los embeddings de un puesto de trabajo
    req_experience (float): experiencia requerida en meses para el puesto de trabajo (valor de referencia para calcular una puntuación entre 0 y 100 en base a diferentes experiencias)
    positions_cap (int, optional): Maximum number of positions to consider for scoring. Defaults to 4.
    min_dist_threshold (float, optional): Distancia entre embeddings a partir de la cual el puesto del CV se considera "equivalente" al de la oferta.
    max_dist_threshold (float, optional): Distancia entre embeddings a partir de la cual el puesto del CV no puntúa.
    
    Returns:
    pandas.DataFrame: DataFrame original añadiendo una columna con las puntuaciones individuales contribuidas por cada puesto.
    float: Puntuación total entre 0 y 100.
    """
    # A efectos de puntuación, computamos para cada puesto como máximo el número total de meses de experiencia requeridos
    df['duration_capped'] = df['duracion'].apply(lambda x: min(x, req_experience))
    # Normalizamos la distancia entre 0 y 1, siendo 0 la distancia mínima y 1 la máxima
    df['adjusted_distance'] = df['distancia'].apply(
        lambda x: 0 if x <= min_dist_threshold else (
            1 if x >= max_dist_threshold else (x - min_dist_threshold) / (max_dist_threshold - min_dist_threshold)
        )
    )
    # Cada puesto puntúa en base a su duración y a la inversa de la distancia (a menor distancia, mayor puntuación)
    df['position_score'] = ((1 - df['adjusted_distance']) * (df['duration_capped']/req_experience) * 100)
    # Descartamos puestos con distancia superior al umbral definido (asignamos puntuación 0), y ordenamos por puntuación
    df.loc[df['distancia'] >= max_dist_threshold, 'position_score'] = 0
    df = df.sort_values(by='position_score', ascending=False)
    # Nos quedamos con los positions_cap puestos con mayor puntuación
    df.iloc[positions_cap:, df.columns.get_loc('position_score')] = 0
    # Totalizamos (no debería superar 100 nunca, pero ponemos un límite para asegurar)
    total_score = min(df['position_score'].sum(), 100)
    return df, total_score

Para entender mejor el algoritmo, podemos probar el currículo de ejemplo para el que habíamos calculado las distancias con el puesto "Cajero supermercado Dia". En su experiencia anterior, veíamos que el puesto más cercano es el de "Vendedor/a de puesto de mercado", pero sólo tiene 6 meses de experiencia. Si probáramos con una experiencia requerida muy alta, como 48 meses, este CV daría una puntuación muy baja. Si, en cambio, el requisito de experiencia es más bajo, el CV obtendrá una puntuación alta gracias a este puesto. Además, los puestos que tienen menor relación semántica con la oferta, pero más meses de experiencia, puntuarán más en función del ajuste de los parámetros de umbral mínimo y máximo de distancia. 

<br>El ajuste fino de los parámetros de umbral mínimo y máximo de distancia de embeddings hace que las experiencias con título más diferente al de la oferta tengan más o menos peso en la puntuación. Estos no son parámetros intuitivos y sólo se pueden ajustar en base a la experiencia: en la aplicación de usuario final, se etiquetarán como "parámetros avanzados" y la recomendación sería encontrar unos valores por defecto "óptimos" en función de la experiencia de múltiples casos de uso. Para este ejemplo, hemos elegido 0.55 y 0.63, dado que sirven para ilustrar muy bien el siguiente ejemplo, si probamos diferentes valores para req_experience (el parámetro positions_cap podemos dejarlo en 4 y no impacta mucho en la puntuación). Estos parámetros se pueden ajustar en función del título de la oferta, quedando fijos para comparar diferentes currículos. **El rango óptimo para los parámetros min_dist_threshold y max_dist_threshold depende funcamentalmente de la longitud del texto de la oferta de trabajo a introducir**. En un entorno real, en el que se evalúen diferentes ofertas, se podrían determinar unos valores "recomendados" de umbrales, pero para este sencillo ejercicio, lógicamente, no disponemos de datos suficientes para realizar ese ajuste fino. 

In [13]:
# Ejemplo de uso con el currículo del notebook anterior
args = [12, 4, 0.55, 0.63] # Argumentos req_experience, positions_cap, min_distance, max_distance
scored_df, total_score = calcula_puntuacion(ejemplos_experiencia_sorted, *args)
print(f"Puntuación: {total_score:.1f}/100")
display(scored_df)

Puntuación: 90.4/100


,empresa,puesto,periodo,fec_inicio,fec_final,duracion,distancia,duration_capped,adjusted_distance,position_score
1,Mercadona,Vendedor/a de puesto de mercado,202310-202404,2023-10-01,2024-04-01,6,0.556915,6,0.086437,45.678127
3,GASTROTEKA ORDIZIA 1990,Camarero/a de barra,202303-202309,2023-03-01,2023-09-01,6,0.587302,6,0.466269,26.686531
2,AGRISOLUTIONS,AUXILIAR DE MANTENIMIENTO INDUSTRIAL,202001-202401,2020-01-01,2024-01-01,48,0.617411,12,0.842632,15.736790
0,Autónomo,Comercial de automoviles,202401,2024-01-01,2024-12-07,11,0.628034,11,0.975419,2.253279
5,Bellota Herramientas,Personal de mantenimiento,202005-202011,2020-05-01,2020-11-01,6,0.647790,6,1.000000,0.000000
4,ZEREGUIN ZERBITZUAK,limpieza industrial,202012-202305,2020-12-01,2023-05-01,29,0.701754,12,1.000000,0.000000


### Ejemplos de puntuación:

Para entender mejor el sistema de puntuación, podemos evaluar diferentes ejemplos en los que el requisito de experiencia sea 100 meses y establezcamos un límite de 4 posiciones a considerar. Los límites de distancia de embeddings no son relevantes en este caso, aunque los elegimos en función de los experimentos realizados anteriormente. Utilizamos los umbrales 0.6 y 0.7 para ilustrar un posible rango razonable de distancias de embeddings para una descripción corta como la utilizada. **El rango óptimo para estos parámetros depende funcamentalmente de la longitud del texto de la oferta de trabajo a introducir**.

In [14]:
args = [100, 4, 0.6, 0.7] # req_experience, positions_cap, min_distance, max_distance

4 experiencias en puesto muy similar al ofertado, sumando 99 meses:

In [15]:
data = [
    {'duracion': 25, 'distancia': 0.6},
    {'duracion': 25, 'distancia': 0.6},
    {'duracion': 25, 'distancia': 0.6},
    {'duracion': 24, 'distancia': 0.6},
    {'duracion': 23, 'distancia': 0.6} # Esta última posición no cuenta, al poner un límite de 4 y ser la de menor puntuación
]

df_very_high_score = pd.DataFrame(data)
scored_df, total_score = calcula_puntuacion(df_very_high_score, *args)
print(f"Total Score: {total_score:.2f}")
display(scored_df)

Total Score: 99.00


,duracion,distancia,duration_capped,adjusted_distance,position_score
0,25,0.6,25,0,25.0
1,25,0.6,25,0,25.0
2,25,0.6,25,0,25.0
3,24,0.6,24,0,24.0
4,23,0.6,23,0,0.0


4 experiencias en puestos menos similares al ofertado, sumando 100 meses:

In [16]:
data = [
    {'duracion': 25, 'distancia': 0.61},
    {'duracion': 25, 'distancia': 0.61},
    {'duracion': 25, 'distancia': 0.61},
    {'duracion': 25, 'distancia': 0.61},
    {'duracion': 25, 'distancia': 0.62} # Esta última posición no cuenta, al poner un límite de 4 y ser la de menor puntuación
]

df_high_score = pd.DataFrame(data)
scored_df, total_score = calcula_puntuacion(df_high_score, *args)
print(f"Total Score: {total_score:.2f}")
display(scored_df)

Total Score: 90.00


,duracion,distancia,duration_capped,adjusted_distance,position_score
0,25,0.61,25,0.1,22.5
1,25,0.61,25,0.1,22.5
2,25,0.61,25,0.1,22.5
3,25,0.61,25,0.1,22.5
4,25,0.62,25,0.2,0.0


Una experiencia de 100 meses en un puesto de "distancia intermedia" al ofertado:

In [17]:
data = [
    {'duracion': 100, 'distancia': 0.65},
    {'duracion': 25, 'distancia': 0.7}, # Descartado por distancia
    {'duracion': 25, 'distancia': 0.7}, # Descartado por distancia
    {'duracion': 25, 'distancia': 0.7}, # Descartado por distancia
    {'duracion': 23, 'distancia': 0.7} # Descartado por distancia
]

df_mid_score = pd.DataFrame(data)
scored_df, total_score = calcula_puntuacion(df_mid_score, *args)
print(f"Total Score: {total_score:.2f}")
display(scored_df)

Total Score: 50.00


,duracion,distancia,duration_capped,adjusted_distance,position_score
0,100,0.65,100,0.5,50.0
1,25,0.70,25,1.0,0.0
2,25,0.70,25,1.0,0.0
3,25,0.70,25,1.0,0.0
4,23,0.70,23,1.0,0.0


50 meses en un puesto muy similar y 50 meses en un puesto de "distancia intermedia":

In [18]:
data = [
    {'duracion': 50, 'distancia': 0.6},
    {'duracion': 50, 'distancia': 0.65},
    {'duracion': 25, 'distancia': 0.7}, # Descartado por distancia
    {'duracion': 25, 'distancia': 0.7}, # Descartado por distancia
    {'duracion': 25, 'distancia': 0.7}, # Descartado por distancia
]

df_mid_high_score = pd.DataFrame(data)
scored_df, total_score = calcula_puntuacion(df_mid_high_score, *args)
print(f"Total Score: {total_score:.2f}")
display(scored_df)

Total Score: 75.00


,duracion,distancia,duration_capped,adjusted_distance,position_score
0,50,0.60,50,0.0,50.0
1,50,0.65,50,0.5,25.0
2,25,0.70,25,1.0,0.0
3,25,0.70,25,1.0,0.0
4,25,0.70,25,1.0,0.0


## 4. Llamada al modelo para generar el fichero JSON final de salida

El último paso, una vez extraídos los datos y calculadas las puntuaciones, será llamar al modelo para que genere un fichero JSON de salida con la siguiente información:

- Puntuación total.
- Listado de experiencias relevantes.
- Descripción de la experiencia.

Los dos primeros elementos se calculan mediante la inferencia de reconocimiento de entidades nombradas del notebook 01, y los cálculos con embeddings de este notebook. Para obetener la salida estructurada completa, haremos una nueva llamada a un modelo gpt en la que le pasaremos la puntuación y la tabla de datos completa, para que elabore un texto explicativo y coherente con los datos calculados. En el siguiente notebook, ejecutaremos el proceso completo para el CV de ejemplo con el que hemos estado trabajando.